In [13]:
from datetime import datetime 
import time


In [ ]:

# Initialize the previous minute
previous_minute = datetime.now().minute

while True:
    # Get the current minute
    current_minute = datetime.now().minute
    
    # Check if the minute has changed
    if current_minute != previous_minute:
        print(f"New minute detected: {current_minute}")
        previous_minute = current_minute

New minute detected: 30
New minute detected: 31


KeyboardInterrupt: 

In [24]:
from vnstock import Quote

import pandas as pd

In [29]:
quote = Quote(symbol='VCI', source='VCI')

try:
    df = quote.history(start='2020-01-01',
                        end='2025-10-05')
    
    print("Trích xuất dữ liệu thành công.")

except (ValueError, RetryError):
    print(f"Không tìm thấy dữ liệu cho VCI.")
    # tạo DataFrame rỗng có cấu trúc chuẩn
    df = pd.DataFrame(columns=["time", "open", "high", "low", "close", "volume"])

df.to_csv("batch_pipeline/storage/VCI.csv")


Trích xuất dữ liệu thành công.


In [31]:
import ta

df['SMA_20'] = ta.trend.sma_indicator(df['close'], window=20).round(4)
df['EMA_20'] = ta.trend.ema_indicator(df['close'], window=20).round(4)

df.to_csv("batch_pipeline/storage/VCI.csv")

In [19]:
from tenacity import RetryError
import pandas as pd


quote = Quote(symbol='VCI', source='VCI')

try:
    df = quote.history(start='2025-10-04', end='2025-10-05')
        
    print("Trích xuất dữ liệu thành công.")

except (ValueError, RetryError):
    print(f"Không tìm thấy dữ liệu cho VCI.")
    # tạo DataFrame rỗng có cấu trúc chuẩn
    df = pd.DataFrame(columns=["time", "open", "high", "low", "close", "volume"])

Không tìm thấy dữ liệu cho VCI.


In [22]:
len(df)

0

In [ ]:
def calculate_metrics(data):
    last_close = data['close'].iloc[-1]
    prev_close = data['close'].iloc[-2]
    change = last_close - prev_close
    pct_change = (change / prev_close) * 100
    high = data['high'].max()
    low = data['low'].min()
    volume = data['volume'].sum()

    return last_close, change, pct_change, high, low, volume

In [8]:
try:
    last_close, change, pct_change, high, low, volume = calculate_metrics(df)
except:
    print("Chưa có dữ liệu hiện tại")


Chưa có dữ liệu hiện tại


In [57]:
listing = Listing(source='VCI')

In [58]:
listing.all_symbols()

,symbol,organ_name
0,YTC,Công ty Cổ phần Xuất nhập khẩu Y tế Thành phố ...
1,YEG,Công ty Cổ phần Tập đoàn Yeah1
2,YBM,Công ty Cổ phần Khoáng sản Công nghiệp Yên Bái
3,YBC,Công ty Cổ phần Xi măng và Khoáng sản Yên Bái
4,XPH,Công ty Cổ phần Xà phòng Hà Nội
...,...,...
1714,AAS,Công ty Cổ phần Chứng khoán SmartInvest
1715,AAM,Công ty Cổ Phần Thủy Sản MeKong
1716,AAH,Công ty Cổ phần Hợp Nhất
1717,AAA,Công ty Cổ phần Nhựa An Phát Xanh


In [59]:
symbol = listing.all_symbols()[['symbol']]

In [93]:
symbol

,symbol
0,YTC
1,YEG
2,YBM
3,YBC
4,XPH
...,...
1714,AAS
1715,AAM
1716,AAH
1717,AAA


In [95]:
for i in symbol:
    print(i)

symbol


In [60]:
symbol.to_csv('symbol.csv')

In [9]:
listing.symbols_by_exchange().head(1719)

,symbol,exchange,type,organ_short_name,organ_name,product_grp_id
0,YTC,UPCOM,STOCK,XNK Y tế TP.HCM,Công ty Cổ phần Xuất nhập khẩu Y tế Thành phố ...,UPX
1,YEG,HSX,STOCK,Tập đoàn Yeah1,Công ty Cổ phần Tập đoàn Yeah1,STO
2,YBM,HSX,STOCK,Khoáng sản CN Yên Bái,Công ty Cổ phần Khoáng sản Công nghiệp Yên Bái,STO
3,YBC,UPCOM,STOCK,Xi măng và Khoáng sản Yên Bái,Công ty Cổ phần Xi măng và Khoáng sản Yên Bái,UPX
4,XPH,UPCOM,STOCK,Xà phòng Hà Nội,Công ty Cổ phần Xà phòng Hà Nội,UPX
...,...,...,...,...,...,...
1714,CVPB2310,DELISTED,CW,CVPB2310 - Chứng quyền,CVPB2310 - Chứng quyền,None
1715,CVPB2309,DELISTED,CW,CVPB2309 - Chứng quyền,CVPB2309 - Chứng quyền,None
1716,CVPB2308,DELISTED,CW,CVPB2308 - Chứng quyền,CVPB2308 - Chứng quyền,None
1717,CVPB2307,DELISTED,CW,CVPB2307 - Chứng quyền,CVPB2307 - Chứng quyền,None


In [92]:
from vnstock import Quote
quote = Quote(symbol='YEG', source='VCI')

quote.history(start='2025-10-01', end='2025-10-2')

,time,open,high,low,close,volume
0,2025-10-01,14.4,14.5,14.20,14.35,1646400
1,2025-10-02,14.5,14.6,14.15,14.20,1097200


In [ ]:
quote = Quote(symbol='YEG', source='VCI')
df = quote.history(start='2025-10-2', end='2025-10-3', interval='1m')

df

,time,open,high,low,close,volume
0,2025-10-02 09:15:00,14.50,14.50,14.50,14.50,17100
1,2025-10-02 09:16:00,14.45,14.50,14.45,14.50,9600
2,2025-10-02 09:17:00,14.50,14.55,14.50,14.50,9200
3,2025-10-02 09:18:00,14.55,14.55,14.50,14.50,44200
4,2025-10-02 09:19:00,14.50,14.55,14.50,14.55,13300
...,...,...,...,...,...,...
156,2025-10-02 14:26:00,14.20,14.20,14.15,14.20,41400
157,2025-10-02 14:27:00,14.15,14.15,14.15,14.15,16600
158,2025-10-02 14:28:00,14.15,14.15,14.15,14.15,13000
159,2025-10-02 14:29:00,14.15,14.15,14.10,14.15,12100


In [91]:
df

,time,open,high,low,close,volume
0,2025-10-02 09:15:00,14.50,14.50,14.50,14.50,17100
1,2025-10-02 09:16:00,14.45,14.50,14.45,14.50,9600
2,2025-10-02 09:17:00,14.50,14.55,14.50,14.50,9200
3,2025-10-02 09:18:00,14.55,14.55,14.50,14.50,44200
4,2025-10-02 09:19:00,14.50,14.55,14.50,14.55,13300
...,...,...,...,...,...,...
117,2025-10-02 13:38:00,14.35,14.35,14.35,14.35,45000
118,2025-10-02 13:39:00,14.30,14.30,14.30,14.30,2200
119,2025-10-02 13:40:00,14.30,14.30,14.30,14.30,1700
120,2025-10-02 13:41:00,14.30,14.30,14.30,14.30,5000


In [85]:
df.to_csv('YEG.csv')

In [ ]:
symbol

,symbol
0,YTC
1,YEG
2,YBM
3,YBC
4,XPH
...,...
1714,AAS
1715,AAM
1716,AAH
1717,AAA


In [11]:
quote = Quote(symbol='VCI', source='VCI')

df = quote.history(start='2025-10-2', end='2025-10-3')

df = df[['time', 'open', 'close']]

df

,time,open,close
0,2025-10-02,43.2,42.1


In [27]:
df['time'].dt.strftime("%Y").iloc[0]

'2025'

In [86]:
for i in symbol.values:
    quote = Quote(symbol=i, source='VCI')

    quote.history(start='2025-10-2', end='2025-10-3', interval='1m')

ValueError: Symbol phải là một chuỗi.

In [44]:
import pandas as pd 

sb = pd.read_csv("batch_pipeline/symbol.csv")

sb = sb[["symbol"]]

In [45]:
sb = sb.iloc[:, 0].tolist()


In [47]:
print('VCI' in sb)

True


In [3]:
from datetime import *

today = date.today()

today.strftime('%Y-%m-%d')

'2025-10-03'

In [6]:
from dateutil.relativedelta import relativedelta

three_yrs_ago = datetime.now() - relativedelta(years=3)

three_yrs_ago.strftime('%Y-%m-%d')

'2022-10-03'

In [7]:
datetime.now()

datetime.datetime(2025, 10, 3, 9, 1, 36, 260701)

In [28]:
for i in range(10):
    print(i)

0
1
2
3
4
5
6
7
8
9


In [ ]:
def transform_data(df):
    df = df[['time', 'open', 'close']]

    return df

def load_data(df, symbol):
    df.to_csv("VCI.csv", index=False)

def extract_data():
    today = date.today()
    five_yrs_from_today = today - relativedelta(years=5)

    quote = Quote(symbol='VCI', source='VCI')

    df = quote.history(start=five_yrs_from_today.strftime('%Y-%m-%d'),
                        end=today.strftime('%Y-%m-%d'))

    transform_data(df)

    load_data(df, 'VCI')


extract_data()

In [48]:
quote = Quote(symbol='VCI', source='VCI')

df = quote.history(start='2025-10-03',
                    end='2025-10-04', interval='1m')

In [50]:
df['time'] = df["time"].dt.strftime("%H:%M")

In [51]:
df['time']

0      09:15
1      09:16
2      09:17
3      09:18
4      09:19
       ...  
129    11:25
130    11:26
131    11:27
132    11:28
133    11:29
Name: time, Length: 134, dtype: object

In [54]:
df.iloc[0]

time      09:15
open       42.1
high       42.1
low        42.0
close      42.0
volume    77600
Name: 0, dtype: object

In [1]:
from vnstock import Quote 

import pandas as pd 
from datetime import date

from tenacity import RetryError
import os 
import ta



today = date.today()

quote = Quote(symbol='VCI', source='VCI')

try:
    df = quote.history(start='2020-01-01',
                        end=today.strftime('%Y-%m-%d'))
    print("Trích xuất dữ liệu thành công")

except (ValueError, RetryError):
    print(f"Không tìm thấy dữ liệu cho {symbol}.")
    # tạo DataFrame rỗng có cấu trúc chuẩn
    df = pd.DataFrame(columns=["time", "open", "high", "low", "close", "volume"])
    df['time'] = pd.Series([today.strftime('%Y-%m-%d')])
    df['time'] = pd.to_datetime(df['time']) 

Trích xuất dữ liệu thành công


In [ ]:
def check_legit_symbol(symbol):
    symbols = pd.read_csv('symbol.csv')[["symbol"]].iloc[:, 0].tolist()

    return symbol in symbols






False